In [33]:
import ollama
print("Environment is working!")


Environment is working!


In [34]:
!pip install ollama



[notice] A new release of pip is available: 25.3 -> 26.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [35]:
import ollama
import json
import re


In [36]:
def extract_json(text):
    json_match = re.search(r'\{.*\}', text, re.DOTALL)
    if json_match:
        return json.loads(json_match.group())
    else:
        raise ValueError("No valid JSON found in model response")


In [37]:

course_name = "web development"
print(f"\nSelected Course: {course_name}")



Selected Course: web development


In [38]:
def generate_initial_mcq(course):
    prompt = f"""
    Generate 5 MCQ questions for the course: {course}

    Rules:
    Q1: Study method preference (Reading / Watching Video / Voice Conversation)
    Q2: Self knowledge level (Beginner / Intermediate / Advanced)
    Q3-5: Basic conceptual questions from the course
    Q3-5 must include correct_answer field.

    Return strictly in JSON format:

    {{
      "questions": [
        {{
          "question": "",
          "options": ["A", "B", "C", "D"],
          "correct_answer": ""
        }}
      ]
    }}
    """

    response = ollama.chat(
        model="llama3:8b-instruct-q4_K_M",
        messages=[{"role": "user", "content": prompt}]
    )

    return extract_json(response['message']['content'])


mcq_data = generate_initial_mcq(course_name)

for i, q in enumerate(mcq_data["questions"], 1):
    print(f"\nQ{i}: {q['question']}")
    for opt in q["options"]:
        print(opt)



Q1: What is your preferred study method?
A) Reading
B) Watching Video
C) Voice Conversation
D) All of the above

Q2: What is your level of self-knowledge in web development?
A) Beginner
B) Intermediate
C) Advanced
D) None

Q3: What does HTML stand for?
A) Hyper Text Markup Language
B) Hypertext Makeup Language
C) Home Tool Markup Language
D) Hidden Text Magic Language

Q4: Which programming language is used for backend development in web development?
A) Java
B) Python
C) JavaScript
D) C++

Q5: What is the primary purpose of a CSS file in web development?
A) To write server-side code
B) To add images to a webpage
C) To style and layout a webpage
D) To create a database


In [39]:
user_answers = []

for i in range(5):
    if i >= 2:
        ans = input(f"Answer for Q{i+1} (or type SKIP): ").strip()
    else:
        ans = input(f"Answer for Q{i+1}: ").strip()
    user_answers.append(ans)

print("\nUser Answers:", user_answers)



User Answers: ['A', 'A', 'a', 'a', 'a']


In [40]:
def evaluate_initial(mcq_data, answers):
    prompt = f"""
    Evaluate the user answers.

    Questions:
    {mcq_data}

    User Answers:
    {answers}

    Instructions:
    - Detect selected study method from Q1
    - Detect knowledge level from Q2
    - Score only Q3-5 (ignore skipped)
    - Identify weak areas from wrong answers

    Return strictly JSON:

    {{
      "study_method": "",
      "knowledge_level": "",
      "score": "",
      "weak_areas": []
    }}
    """

    response = ollama.chat(
        model="llama3:8b-instruct-q4_K_M",
        messages=[{"role": "user", "content": prompt}]
    )

    return extract_json(response['message']['content'])


evaluation = evaluate_initial(mcq_data, user_answers)

print("\n=== Evaluation Result ===")
print(json.dumps(evaluation, indent=4))



=== Evaluation Result ===
{
    "study_method": "A) Reading",
    "knowledge_level": "Beginner",
    "score": {
        "HTML": {
            "correct": true
        },
        "Backend programming language": {
            "correct": false
        },
        "Primary purpose of CSS file": {
            "correct": false
        }
    },
    "weak_areas": [
        "Programming languages (Backend development)",
        "CSS fundamentals"
    ]
}


In [42]:
def generate_roadmap(course, evaluation):
    prompt = f"""
    Create a personalized roadmap for the course: {course}

    Based on:
    {evaluation}

    Rules:
    - Topic 1 must be common introduction for all learners
    - Remaining topics must adapt to knowledge level
    - If weak_areas exist, add reinforcement topics
    - No static roadmap
    - No placeholders
    - Return strictly JSON:

    {{
      "topics": [
        {{
          "topic_name": "",
          "level": ""
        }}
      ]
    }}
    """

    response = ollama.chat(
        model="llama3:8b-instruct-q4_K_M",
        messages=[{"role": "user", "content": prompt}]
    )

    return extract_json(response['message']['content'])


roadmap = generate_roadmap(course_name, evaluation)

print("\n=== Personalized Roadmap ===")
for i, topic in enumerate(roadmap["topics"], 1):
    print(f"{i}. {topic['topic_name']} ({topic['level']})")



=== Personalized Roadmap ===
1. Web Development Basics (Beginner)
2. HTML Fundamentals (Beginner)
3. CSS Fundamentals (Beginner)
4. Backend Programming Language Basics (Intermediate)
5. Reinforcing CSS Fundamentals (Beginner)


In [43]:
def generate_topic_content(course, topic_name, study_method):
    prompt = f"""
    Generate detailed educational content for:

    Course: {course}
    Topic: {topic_name}
    Study Method: {study_method}

    Requirements:
    - Paragraph wise explanation
    - Clear conceptual flow
    - Beginner-friendly if needed
    - No bullet list only paragraphs
    """

    response = ollama.chat(
        model="llama3:8b-instruct-q4_K_M",
        messages=[{"role": "user", "content": prompt}]
    )

    return response['message']['content']


In [44]:
print("\n=== Generating Topic 1 Content ===")

topic1 = roadmap["topics"][0]["topic_name"]
study_method = evaluation["study_method"]

content = generate_topic_content(course_name, topic1, study_method)

print(f"\n=== {topic1} ===\n")
print(content)



=== Generating Topic 1 Content ===

=== Web Development Basics ===

**Web Development Basics**

### What is Web Development?

Before we dive into the world of web development, let's first understand what it entails. Web development refers to the process of building a website or web application that can be accessed over the internet. This involves creating the front-end user interface and back-end logic that enables users to interact with the site and retrieve data from servers. As a beginner in web development, you'll learn about the different languages, tools, and technologies used to build dynamic websites.

### HTML: The Building Blocks of Web Development

HTML (Hypertext Markup Language) is the most fundamental language of the web. It's used to create the structure and content of a webpage by defining elements such as headings, paragraphs, images, links, and more. HTML consists of a series of tags that wrap around content to define its purpose. For instance, the `<h1>` tag defines

In [45]:
def generate_topic_quiz(topic_content):
    prompt = f"""
    Generate 5 MCQ questions based only on this content:

    {topic_content}

    Return strictly JSON:

    {{
      "questions": [
        {{
          "question": "",
          "options": ["A", "B", "C", "D"],
          "correct_answer": ""
        }}
      ]
    }}
    """

    response = ollama.chat(
        model="llama3:8b-instruct-q4_K_M",
        messages=[{"role": "user", "content": prompt}]
    )

    return extract_json(response['message']['content'])


In [46]:
def evaluate_topic_quiz(quiz_data, user_answers):
    correct = 0
    weak = []

    for i, q in enumerate(quiz_data["questions"]):
        if user_answers[i].strip().upper() == q["correct_answer"].strip().upper():
            correct += 1
        else:
            weak.append(q["question"])

    score = f"{correct}/5"

    return {
        "score": score,
        "weak_areas": weak
    }


In [47]:
def refine_remaining_roadmap(course, remaining_topics, weak_areas):
    prompt = f"""
    Refine only the remaining roadmap topics for the course: {course}

    Remaining Topics:
    {remaining_topics}
    Weak Areas (convert these into proper topic names before adding):
    {weak_areas}

    

    Instructions:
    - Do NOT include completed topics
    - Do not use full question sentences as topic names.
    - Convert them into proper concise topic titles.
    - Add reinforcement topic if weak_areas exist
    - Maintain logical order
    - Return strictly JSON:

    {{
      "topics": [
        {{
          "topic_name": "",
          "level": ""
        }}
      ]
    }}
    """

    response = ollama.chat(
        model="llama3:8b-instruct-q4_K_M",
        messages=[{"role": "user", "content": prompt}]
    )

    return extract_json(response['message']['content'])


In [48]:
completed_topics = []

topics_list = roadmap["topics"]

while topics_list:

    current_topic = topics_list[0]
    topic_name = current_topic["topic_name"]

    print(f"\n=== Studying: {topic_name} ===\n")

    # Generate Content
    content = generate_topic_content(course_name, topic_name, evaluation["study_method"])
    print(content)

    # Generate Quiz
    quiz = generate_topic_quiz(content)

    print("\n=== Topic Quiz ===")
    for i, q in enumerate(quiz["questions"], 1):
        print(f"\nQ{i}: {q['question']}")
        for opt in q["options"]:
            print(opt)

    # Take Answers
    topic_answers = []
    for i in range(5):
        ans = input(f"Answer for Q{i+1}: ")
        topic_answers.append(ans)

    # Evaluate Quiz
    result = evaluate_topic_quiz(quiz, topic_answers)

    print("\nScore:", result["score"])

    # Mark Topic Completed
    completed_topics.append(topic_name)

    # Remove Completed Topic
    topics_list = topics_list[1:]

    # Refine Remaining Topics
    if topics_list:
        refined = refine_remaining_roadmap(course_name, topics_list, result["weak_areas"])
        topics_list = refined["topics"]

print("\n🎉 Course Completed Successfully!")



=== Studying: Web Development Basics ===

Here is the detailed educational content for the topic "Web Development Basics" using a reading study method:

**What is Web Development?**

Web development refers to the process of designing, building, and maintaining websites. It involves creating interactive and dynamic content that users can access through the internet. A web developer's primary goal is to design and develop a website that meets the client's requirements, is user-friendly, and is compatible with different devices and browsers.

**HTML: The Building Blocks of Web Pages**

HTML (Hypertext Markup Language) is the standard markup language used to create web pages. It is the foundation of any web development project, as it provides the structure and content for a website. HTML consists of a series of elements, such as headings, paragraphs, images, and links, which are represented by tags. These tags are surrounded by angle brackets (<>) and are used to define the different part

JSONDecodeError: Expecting ',' delimiter: line 25 column 51 (char 2011)